In [1]:
import stompy.model.delft.dflow_model as dfm
import numpy as np
import logging
import pandas as pd
from matplotlib import dates

import xarray as xr
from stompy.grid import multi_ugrid
from stompy import utils, memoize, filters
import os
import six
import matplotlib.pyplot as plt
from matplotlib import colors

%matplotlib notebook

In [2]:
import run_database
six.moves.reload_module(run_database)
all_runs=run_database.all_runs

In [3]:
import common
from common import (prechain, his_cache, save_as_layers, load_or_none, name_runs,
                    scen_names, slr_names, events, label_events,
                   ebb_period,flood_period,ebb_time,spring_ebb_mid,
                   spring_flood_mid, spring_ebb, spring_flood)

In [6]:
runs=run_database.select(period='2016long',slr=0.00,flows='impaired')

runs['model']=runs.run_dir.apply(lambda rd: dfm.DFlowModel.load(rd))

name_runs(runs)

In [57]:
for _,run in runs.iterrows():
    his_ds=run['model'].his_dataset(chain=True,prechain=prechain)

    variables=['time','station_x_coordinate','station_y_coordinate',
               'stations', # 'station_id',
               'waterlevel','bedlevel', # 'taus',
               'depth-averaged_x_velocity','depth-averaged_y_velocity', 

               'cross_section',
               'cross_section_name','cross_section_discharge','cross_section_area',
               'cross_section_velocity'
               ]
    stations=['pch_up','pch_down','lag1','nmc_down','BC3','ch2','bc1',
              'nck','bbr','bbrch','pc3','nmp','nmc_up','mid_mouth','mouth_thalweg',
              'npc']

    ds=his_ds.copy() # shallow

    for v in list(ds):
        if v not in variables:
            del ds[v]
    for v in list(ds.coords):
        dims=ds[v].dims
        if v=='time': continue
        if ('laydim' in dims) or ('laydimw' in dims):
            del ds[v]
            continue
        if 'stations' in dims: continue
        if 'cross_section' in dims: continue
        del ds[v]

    del ds['cross_section_name']
    del ds['station_name']

    ds_sel=ds.sel(stations=stations)
    ds_sel.attrs['projection']='EPSG:26910'
    ds_sel.attrs['model_run']=run['run_dir']
    export_dir='export'
    if not os.path.exists(export_dir):
        os.makedirs(export_dir)
    nc_fn=os.path.join(export_dir,f'{run["name"]}.nc')
    print("Compute")
    ds_sel_c=ds_sel.compute()
    print(f"Writing to {nc_fn}")
    ds_sel_c.to_netcdf(nc_fn)

Compute
Writing to export/Base.nc
2 chained datasets
Geometries match?
Compute
Writing to export/Low.nc
3 chained datasets
Geometries match?
Geometries match?
Compute
Writing to export/Medium.nc
2 chained datasets
Geometries match?
Compute
Writing to export/High.nc


In [58]:
ls -lh export

total 48M
-rw-rw-r-- 1 rustyh rustyh 26M Oct 18 19:43 Base.nc
-rw-rw-r-- 1 rustyh rustyh 24M Oct 18 20:33 High.nc
-rw-rw-r-- 1 rustyh rustyh 24M Oct 18 19:59 Low.nc
-rw-rw-r-- 1 rustyh rustyh 24M Oct 18 20:18 Medium.nc
